# Dynamic optimization

## Intro

![title](res/FLOWCHART_videodynopt_v0.1.png)

## Init

In [1]:
# imports
import os #to access system folders
import subprocess #to access ffmpeg in the system
import shutil #to remove directories
import numpy as np #easy vector operations
import math #for operation with infinite
from scipy.optimize import curve_fit #fittin of the curve
import json #to handle json files
import matplotlib.pyplot as pl #to display plots
import tkinter as tk #to import file
from tkinter import filedialog #to open import dialog

#constants
PARAM_AVC = {"crfs": 52, "starting_range": [17,28], "lib": "libx264", "container": "mp4", "add_param": ""}
PARAM_HEVC = {"crfs": 52, "starting_range": [22,33], "lib": "libx265", "container": "mp4", "add_param": ""}
PARAM_VP9 = {"crfs": 64, "starting_range": [15,35], "lib": "libvpx-vp9", "container": "webm", "add_param": "-b:v 0"}

#variables - CUSTOM
codec = "avc" #values: "avc", "hevc", "vp9", !!not implemented: "av1", "vvc"
raw_width = 1920
raw_height = 1080
raw_fps = 29.97
#values "bitrate","vmaf", "psnr" !!not implemented: "ssim", "mssim"
target_list = {"vmaf": [90], "bitrate": []}

#input file
root = tk.Tk()
root.withdraw()
source_path = os.path.relpath(filedialog.askopenfilename())
source_name = os.path.basename(source_path).split('.')[0]
REF_PATH = "test_vids/tempRAW_refs/" #raw files for each shot
[os.remove(REF_PATH+f) for f in os.listdir(REF_PATH)] #clean temp_refs folder
DIST_PATH = "test_vids/temp_encoded/" #encoded files for each shot
[shutil.rmtree(DIST_PATH+f) for f in os.listdir(DIST_PATH)] #clean temp_encoded folder

#assessment files path
tm_file = "rd_results/template.json"
rd_file = "rd_results/" + source_name + ".json"
if not os.path.isfile(rd_file): #if file does not exists create it
    with open(rd_file, 'w') as f:
        pass
VMAF_LOGS = "rd_results/vmaf_logs"

#shot detection
TIME_LOGS = "shot_detection.log"
shot_th = 0.25 #change shot threshold
num_scenes = 0

#output file
OUT_LIST = "shot_list.txt"
OUT_PATH = "test_vids/OPT_vids/"

current_point = None #the current optimum point
new_point = 0 #new value to compare with current_point
#all computed points, by row: crf, bitrate, vmaf, psnr
res_matrix = {"crf": None, "bitrate": None, "vmaf": None, "psnr": None} 

print("init done")

init done


## Shot change detection

In [2]:
#an empty json structure is generated to be filled and to store computed values
def init_res_matrix(x):
    res_matrix["crf"] = np.arange(0,x,1).tolist()
    inf_matrix = np.zeros(x) #infinity to avoid considering zero as a point
    inf_matrix[inf_matrix == 0] = math.inf
    res_matrix["bitrate"] = inf_matrix.tolist()
    res_matrix["vmaf"] = inf_matrix.tolist()
    res_matrix["psnr"] = inf_matrix.tolist()
    
#detect shot changes in the scene and split it into shots
def shot_change_detection(p):
    start_t = 0.0
    end_t = 0.0
    #return when the shot changes
    det = f"ffmpeg -i {p} -filter_complex:v \"select='gt(scene,{shot_th})', \
        metadata=print:file={TIME_LOGS}\" -f null -"
    subprocess.call(det, shell=True)
    #get the total duration for the last cut
    idu = f"ffprobe -v error -select_streams v:0 -show_entries format:stream -print_format json {p}"
    dta = json.loads(subprocess.run(idu.split(), stdout=subprocess.PIPE, stderr=subprocess.STDOUT).stdout)
    duration = dta['format']['duration']
    
    with open("shot_detection.log", 'r') as r:
        tm_log = r.read().splitlines()[::2]
    tm_log.append("end pts_time:" + str(duration))
    n = len(tm_log)
    for i,l in enumerate(tm_log): #for each cut
        #create a folder for each scene
        new_dir = str(i)
        new_path = os.path.join(DIST_PATH, new_dir)
        os.mkdir(new_path)
        
        #cut the video
        end_t = l.split("pts_time:",1)[1]
        cut = f"ffmpeg -ss {start_t} -to {end_t} -i {p} \
            -pix_fmt yuv420p {REF_PATH}scene{str(i).zfill(7)}.yuv"
        subprocess.call(cut, shell=True)
        start_t = end_t
    return n

In [3]:
struct_points = [] #structure of target points for the json file
struct_shots = [] #structure of shots for the json file

if source_path.endswith(".yuv"):
    print("yuv input")
elif source_path.endswith(".y4m"):
    print("y4m input")
else:
    print("No such an input type")
    exit()

num_scenes = shot_change_detection(source_path)
    
#init values based on the selected output codec
if codec == "avc":
    s_cod = PARAM_AVC
    init_res_matrix(PARAM_AVC["crfs"])
elif codec == "hevc":
    s_cod = PARAM_HEVC
    init_res_matrix(PARAM_HEVC["crfs"])
elif codec == "vp9":
    s_cod = PARAM_VP9
    init_res_matrix(PARAM_VP9["crfs"])
else:
    print("No such an codec")
    exit()

min_range_crf = s_cod["starting_range"][0]
max_range_crf = s_cod["starting_range"][1]

with open(tm_file, 'r') as f:
    o_data = json.load(f)
    
    #add source name and results matrix
    o_data["content"] = source_name
    o_data["versions"][0]["codec"] = codec
    o_data["versions"][0]["width"] = raw_width
    o_data["versions"][0]["height"] = raw_height
    o_data["versions"][0]["fps"] = raw_fps
    o_data["versions"][0]["shots"][0]["assessment"] = res_matrix
    
    #add emplty target points
    base_point = o_data["versions"][0]["shots"][0]["opt_points"][0]
    for t_name in target_list:
        for t_val in target_list[t_name]:
            base_point["metric"] = t_name
            base_point["target"] = t_val
            struct_points.append(base_point.copy())
    o_data["versions"][0]["shots"][0]["opt_points"] = struct_points
    
    #add empty shots
    base_shot = o_data["versions"][0]["shots"][0]
    for i in range(0, num_scenes):
        base_shot["index"] = i #assign index to shots in json file
        struct_shots.append(base_shot.copy())
    o_data["versions"][0]["shots"] = struct_shots

with open(rd_file, 'w') as w:
    json.dump(o_data, w, separators=(',',': '))

y4m input


ffmpeg version N-106635-g83e1a1de88 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/home/ubuntu/ffmpeg_build --pkg-config-flags=--static --extra-cflags=-I/home/ubuntu/ffmpeg_build/include --extra-ldflags=-L/home/ubuntu/ffmpeg_build/lib --extra-libs='-lpthread -lm' --ld=g++ --bindir=/home/ubuntu/bin --enable-gpl --enable-gnutls --enable-libaom --enable-libass --enable-libfdk-aac --enable-libfreetype --enable-libmp3lame --enable-libopus --enable-libsvtav1 --enable-libdav1d --enable-libvorbis --enable-libvpx --enable-libx264 --enable-libx265 --enable-libvmaf --enable-version3 --enable-nonfree
  libavutil      57. 24.101 / 57. 24.101
  libavcodec     59. 26.100 / 59. 26.100
  libavformat    59. 22.100 / 59. 22.100
  libavdevice    59.  6.100 / 59.  6.100
  libavfilter     8. 33.100 /  8. 33.100
  libswscale      6.  6.100 /  6.  6.100
  libswresample   4.  6.100 /  4.  6.100
  libpostproc    56.  5.100 / 56.  5.100

frame=   90 fps=0.0 q=-0.0 Lsize=  273375kB time=00:00:03.00 bitrate=745750.2kbits/s speed=10.5x    
video:273375kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.000000%


## Optimization
Find the shot encoded to a certain crf that has the closest quality or rate to the target

In [4]:
def encode(s,i,p):
    add_info = s_cod["add_param"]
    lib = s_cod["lib"]
    o = DIST_PATH + str(i) + "/" + str(p) + "_" + codec.upper() + "." + s_cod["container"]
    enc = f"ffmpeg -f rawvideo -video_size {raw_width}x{raw_height} -r {raw_fps} \
        -pixel_format yuv420p -i {REF_PATH+s} -c:v {lib} -crf {new_point} {add_info} {o}"
    subprocess.call(enc, shell=True)
    return o

def assess(s,o):
    c_vmaf = f"ffmpeg -f rawvideo -r {raw_fps} -video_size {raw_width}x{raw_height} -i {REF_PATH+s} \
            -i {o} \
            -lavfi \"[0:v]setpts=PTS-STARTPTS[ref];\
                    [1:v]scale={raw_width}x{raw_height}:flags=bicubic, setpts=PTS-STARTPTS[dist];\
                    [dist][ref]libvmaf=feature=name=psnr:log_path={VMAF_LOGS}:log_fmt=json\" \
            -f null -" #|name=float_ssim|name=float_ms_ssim to compute the other metrics
    subprocess.call(c_vmaf, shell=True)

#store the quality and rate results for each shot at each encoded crf
def save_results(i, c, o):
    with open(VMAF_LOGS, 'r') as r: #extract quality and rate values
        i_data = json.load(r)
        vmaf = i_data["pooled_metrics"]["vmaf"]["mean"]
        psnr = (6*i_data["pooled_metrics"]["psnr_y"]["mean"] + \
            i_data["pooled_metrics"]["psnr_cb"]["mean"] + i_data["pooled_metrics"]["psnr_cr"]["mean"])/8
        info = f"ffprobe -v error -select_streams v:0 -show_entries format:stream -print_format json {o}"
        cout = json.loads(subprocess.run(info.split(), stdout=subprocess.PIPE, stderr=subprocess.STDOUT).stdout)
        bitrate = int(cout['format']['bit_rate'])
        
    with open(rd_file, 'r') as f: #add new values
        o_data = json.load(f)
        o_data["versions"][0]["shots"][i]["assessment"]["crf"][c] = c
        o_data["versions"][0]["shots"][i]["assessment"]["bitrate"][c] = bitrate
        o_data["versions"][0]["shots"][i]["assessment"]["vmaf"][c] = vmaf
        o_data["versions"][0]["shots"][i]["assessment"]["psnr"][c] = psnr
    with open(rd_file, 'w') as w:
        json.dump(o_data, w, separators=(',',': '))
    return o_data["versions"][0]["shots"][i]["assessment"]

def search_opt(t_name, t_val, sx_end, dx_end, i_sx_end, i_dx_end):
    if t_val < sx_end and t_val > dx_end: #if the target is in the range
        n = interpolate(i_sx_end, i_dx_end)
    elif t_val > sx_end: #if the target is out of the range in the left side
        n = 0 #if no other points in this direction had been stored encode at the min crf
        i = i_sx_end - 1
        while n == 0 and i>= 0:
        #the first point you find is the new lower end of the range
            if not res_matrix[t_name][i] == math.inf:
                n = interpolate(i, i_sx_end)
            i -= 1
    elif t_val < dx_end: #if the target is out of the range in the right side
        n = s_cod["crfs"]-1 #if there's no other points, encode at the max crf
        i = i_dx_end + 1
        while n == s_cod["crfs"]-1 and i <= s_cod["crfs"]-1:
            #the first point you find is the new upper end of the range
            if not res_matrix[t_name][i] == math.inf:
                n = interpolate(i_dx_end, i)
            i += 1
    return n

def save_opt(i, t, opt):
    with open(rd_file, 'r') as f:
        o_data = json.load(f)
    o_data["versions"][0]["shots"][i]["opt_points"][t]["crf"] = int(opt)
    with open(rd_file, 'w') as w:
        json.dump(o_data, w, separators=(',',': '))

#linear interpolation of the target and the weight alpha between sx and dx
def interpolate(sx, dx):
    alpha = (res_matrix[t_name][sx] - t_val) / (res_matrix[t_name][sx] - res_matrix[t_name][dx])
    new_point = round(res_matrix["crf"][sx] - alpha * (res_matrix["crf"][sx] - res_matrix["crf"][dx]))
    return new_point

In [5]:
shot_index = 0
point_index = 0
target_index = 0

for shot in sorted(os.listdir(REF_PATH)): #for each shot
    print("init computing -scene" + str(shot_index))
    for t_name in target_list:
        for t_val in target_list[t_name]:
            while not current_point == new_point: #if no convergence
                if not point_index == 0 : # same shot but new target: new_crf before encoding
                    out = encode(shot, shot_index, new_point) #encoding
                    assess(shot, out) #quality assessment
                    #TODO: results must be weighted based on duration (?)
                    res_matrix = save_results(shot_index, new_point, out)
                    
                if point_index == 0: #if there are no points (first loop)
                    new_point = max_range_crf #encode at the upper value in the starting range
                elif point_index == 1: #if there are no points to compare (second loop)
                    current_point = max_range_crf #the current optimal point is the first one
                    new_point = min_range_crf #in the next loop encode at the lower end of the starting range
                else:
                    #element-wise difference between the metric and its target value
                    difference = np.asarray(abs(np.asarray(res_matrix[t_name]) - t_val))
                    #the minimum difference = the element with the index closer to the target
                    i_first_min = np.argmin(difference)
                    nd_diff = difference.copy()
                    nd_diff[i_first_min] = np.inf #replace the minimum with inf
                    i_second_min = np.argmin(nd_diff) #find the second minimum
                    current_point = i_first_min #the index of the point closer to the target

                    if res_matrix[t_name][i_first_min] == res_matrix[t_name][i_second_min]:
                        #it may happen that the results are the same ex. black short shots
                        current_point = max(i_first_min,i_second_min)
                        new_point = current_point
                    else: #swap the values of the two ends if the lower end is bigger than the upper end
                        if res_matrix[t_name][i_first_min] > res_matrix[t_name][i_second_min]:
                            new_point = search_opt(t_name, t_val, res_matrix[t_name][i_first_min], \
                                        res_matrix[t_name][i_second_min], i_first_min, i_second_min)
                        elif res_matrix[t_name][i_first_min] < res_matrix[t_name][i_second_min]:
                            new_point = search_opt(t_name, t_val, res_matrix[t_name][i_second_min], \
                                        res_matrix[t_name][i_first_min], i_second_min, i_first_min)
                point_index += 1
            save_opt(shot_index, target_index, current_point)
            target_index += 1
            current_point = None
            new_point = 0
    shot_index += 1
    point_index = 0
    target_index = 0

ffmpeg version N-106635-g83e1a1de88 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/home/ubuntu/ffmpeg_build --pkg-config-flags=--static --extra-cflags=-I/home/ubuntu/ffmpeg_build/include --extra-ldflags=-L/home/ubuntu/ffmpeg_build/lib --extra-libs='-lpthread -lm' --ld=g++ --bindir=/home/ubuntu/bin --enable-gpl --enable-gnutls --enable-libaom --enable-libass --enable-libfdk-aac --enable-libfreetype --enable-libmp3lame --enable-libopus --enable-libsvtav1 --enable-libdav1d --enable-libvorbis --enable-libvpx --enable-libx264 --enable-libx265 --enable-libvmaf --enable-version3 --enable-nonfree
  libavutil      57. 24.101 / 57. 24.101
  libavcodec     59. 26.100 / 59. 26.100
  libavformat    59. 22.100 / 59. 22.100
  libavdevice    59.  6.100 / 59.  6.100
  libavfilter     8. 33.100 /  8. 33.100
  libswscale      6.  6.100 /  6.  6.100
  libswresample   4.  6.100 /  4.  6.100
  libpostproc    56.  5.100 / 56.  5.100

init computing -scene0


frame=  318 fps= 22 q=-1.0 Lsize=    7982kB time=00:00:10.51 bitrate=6221.0kbits/s speed=0.742x      
video:7977kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.058664%
[libx264 @ 0x5641f4faa180] frame I:2     Avg QP:29.81  size:159704
[libx264 @ 0x5641f4faa180] frame P:79    Avg QP:31.53  size: 58639
[libx264 @ 0x5641f4faa180] frame B:237   Avg QP:35.06  size: 13569
[libx264 @ 0x5641f4faa180] consecutive B-frames:  0.6%  0.0%  0.0% 99.4%
[libx264 @ 0x5641f4faa180] mb I  I16..4:  3.1% 81.2% 15.7%
[libx264 @ 0x5641f4faa180] mb P  I16..4:  0.4% 11.4%  1.1%  P16..4: 32.8% 20.4% 12.5%  0.0%  0.0%    skip:21.4%
[libx264 @ 0x5641f4faa180] mb B  I16..4:  0.0%  0.6%  0.0%  B16..8: 33.3%  8.1%  1.7%  direct: 2.7%  skip:53.5%  L0:36.1% L1:51.3% BI:12.6%
[libx264 @ 0x5641f4faa180] 8x8 transform intra:87.8% inter:71.1%
[libx264 @ 0x5641f4faa180] coded y,uvDC,uvAC intra: 76.7% 73.1% 31.2% inter: 15.7% 9.2% 0.3%
[libx264 @ 0x5641f4faa180] i16 v,h,dc,p: 29% 24%  9% 38

Output #0, null, to 'pipe:':
  Metadata:
    encoder         : Lavf59.22.100
  Stream #0:0: Video: wrapped_avframe, yuv420p(progressive), 1920x1080, q=2-31, 200 kb/s, 29.97 fps, 29.97 tbn
    Metadata:
      encoder         : Lavc59.26.100 wrapped_avframe
frame=  318 fps=8.0 q=-0.0 Lsize=N/A time=00:00:10.61 bitrate=N/A speed=0.268x    
video:147kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: unknown
[Parsed_libvmaf_3 @ 0x555f0b89fcc0] VMAF score: 98.609098
ffmpeg version N-106635-g83e1a1de88 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/home/ubuntu/ffmpeg_build --pkg-config-flags=--static --extra-cflags=-I/home/ubuntu/ffmpeg_build/include --extra-ldflags=-L/home/ubuntu/ffmpeg_build/lib --extra-libs='-lpthread -lm' --ld=g++ --bindir=/home/ubuntu/bin --enable-gpl --enable-gnutls --enable-libaom --enable-libass --enable-libfdk-aac --enable-libfreetype --enable-libmp3lame --enable

frame=  318 fps= 19 q=-1.0 Lsize=   12392kB time=00:00:10.51 bitrate=9658.0kbits/s speed=0.637x    
video:12387kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.038024%
[libx264 @ 0x563dee44d180] frame I:2     Avg QP:26.81  size:228015
[libx264 @ 0x563dee44d180] frame P:79    Avg QP:28.44  size: 92806
[libx264 @ 0x563dee44d180] frame B:237   Avg QP:31.79  size: 20657
[libx264 @ 0x563dee44d180] consecutive B-frames:  0.6%  0.0%  0.0% 99.4%
[libx264 @ 0x563dee44d180] mb I  I16..4:  1.1% 91.3%  7.6%
[libx264 @ 0x563dee44d180] mb P  I16..4:  0.2% 15.7%  1.6%  P16..4: 31.7% 22.4% 16.0%  0.0%  0.0%    skip:12.3%
[libx264 @ 0x563dee44d180] mb B  I16..4:  0.0%  0.9%  0.1%  B16..8: 33.7% 10.8%  3.0%  direct: 4.3%  skip:47.2%  L0:34.4% L1:43.0% BI:22.6%
[libx264 @ 0x563dee44d180] 8x8 transform intra:89.9% inter:67.1%
[libx264 @ 0x563dee44d180] coded y,uvDC,uvAC intra: 87.8% 83.5% 43.5% inter: 21.8% 14.5% 0.8%
[libx264 @ 0x563dee44d180] i16 v,h,dc,p: 27% 26%  4% 43

...

frame=  102 fps= 20 q=-1.0 Lsize=    2609kB time=00:00:03.30 bitrate=6469.8kbits/s speed=0.648x    
video:2607kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.079231%
[libx264 @ 0x55fdca6f7180] frame I:1     Avg QP:29.29  size: 67551
[libx264 @ 0x55fdca6f7180] frame P:26    Avg QP:29.78  size: 48208
[libx264 @ 0x55fdca6f7180] frame B:75    Avg QP:32.06  size: 17970
[libx264 @ 0x55fdca6f7180] consecutive B-frames:  2.0%  0.0%  0.0% 98.0%
[libx264 @ 0x55fdca6f7180] mb I  I16..4: 11.7% 78.3%  9.9%
[libx264 @ 0x55fdca6f7180] mb P  I16..4:  1.6% 17.9%  2.6%  P16..4: 44.4% 15.3%  4.8%  0.0%  0.0%    skip:13.4%
[libx264 @ 0x55fdca6f7180] mb B  I16..4:  0.1%  1.0%  0.1%  B16..8: 50.4%  8.8%  1.4%  direct: 2.2%  skip:35.9%  L0:43.6% L1:50.4% BI: 6.0%
[libx264 @ 0x55fdca6f7180] 8x8 transform intra:81.2% inter:83.9%
[libx264 @ 0x55fdca6f7180] coded y,uvDC,uvAC intra: 67.5% 64.7% 26.8% inter: 17.7% 9.8% 0.2%
[libx264 @ 0x55fdca6f7180] i16 v,h,dc,p: 21% 18%  5% 56%


init computing -scene3


ffmpeg version N-106635-g83e1a1de88 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/home/ubuntu/ffmpeg_build --pkg-config-flags=--static --extra-cflags=-I/home/ubuntu/ffmpeg_build/include --extra-ldflags=-L/home/ubuntu/ffmpeg_build/lib --extra-libs='-lpthread -lm' --ld=g++ --bindir=/home/ubuntu/bin --enable-gpl --enable-gnutls --enable-libaom --enable-libass --enable-libfdk-aac --enable-libfreetype --enable-libmp3lame --enable-libopus --enable-libsvtav1 --enable-libdav1d --enable-libvorbis --enable-libvpx --enable-libx264 --enable-libx265 --enable-libvmaf --enable-version3 --enable-nonfree
  libavutil      57. 24.101 / 57. 24.101
  libavcodec     59. 26.100 / 59. 26.100
  libavformat    59. 22.100 / 59. 22.100
  libavdevice    59.  6.100 / 59.  6.100
  libavfilter     8. 33.100 /  8. 33.100
  libswscale      6.  6.100 /  6.  6.100
  libswresample   4.  6.100 /  4.  6.100
  libpostproc    56.  5.100 / 56.  5.100

frame=   90 fps= 12 q=-1.0 Lsize=    9927kB time=00:00:02.90 bitrate=28012.9kbits/s speed=0.385x     
video:9925kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.019955%
[libx264 @ 0x55c40bf7e180] frame I:1     Avg QP:19.87  size:201174
[libx264 @ 0x55c40bf7e180] frame P:23    Avg QP:20.14  size:163589
[libx264 @ 0x55c40bf7e180] frame B:66    Avg QP:21.21  size: 93919
[libx264 @ 0x55c40bf7e180] consecutive B-frames:  2.2%  0.0%  0.0% 97.8%
[libx264 @ 0x55c40bf7e180] mb I  I16..4:  1.0% 94.1%  4.9%
[libx264 @ 0x55c40bf7e180] mb P  I16..4:  0.6% 63.6%  2.4%  P16..4: 16.0% 12.9%  4.4%  0.0%  0.0%    skip: 0.2%
[libx264 @ 0x55c40bf7e180] mb B  I16..4:  0.1% 23.1%  0.5%  B16..8: 34.3% 16.7%  3.9%  direct:10.7%  skip:10.7%  L0:41.3% L1:38.5% BI:20.2%
[libx264 @ 0x55c40bf7e180] 8x8 transform intra:96.4% inter:70.6%
[libx264 @ 0x55c40bf7e180] coded y,uvDC,uvAC intra: 95.0% 92.7% 62.0% inter: 54.8% 53.3% 3.6%
[libx264 @ 0x55c40bf7e180] i16 v,h,dc,p: 22%  3%  0% 7

Output #0, null, to 'pipe:':
  Metadata:
    encoder         : Lavf59.22.100
  Stream #0:0: Video: wrapped_avframe, yuv420p(progressive), 1920x1080, q=2-31, 200 kb/s, 29.97 fps, 29.97 tbn
    Metadata:
      encoder         : Lavc59.26.100 wrapped_avframe
frame=   90 fps=8.2 q=-0.0 Lsize=N/A time=00:00:03.00 bitrate=N/A speed=0.275x    
video:41kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: unknown
[Parsed_libvmaf_3 @ 0x55c701a0e380] VMAF score: 91.769704
ffmpeg version N-106635-g83e1a1de88 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/home/ubuntu/ffmpeg_build --pkg-config-flags=--static --extra-cflags=-I/home/ubuntu/ffmpeg_build/include --extra-ldflags=-L/home/ubuntu/ffmpeg_build/lib --extra-libs='-lpthread -lm' --ld=g++ --bindir=/home/ubuntu/bin --enable-gpl --enable-gnutls --enable-libaom --enable-libass --enable-libfdk-aac --enable-libfreetype --enable-libmp3lame --enable-

## Encode opt video
Put together all the individually encoded shots

In [6]:
def mux(t_i, t_name, t_val):
    file_list = "" #list of encoded vids to be stored in OUT_LIST
    with open(rd_file, 'r') as f:
        o_data = json.load(f)
    for shot in range(0,num_scenes):
        opt_crf = o_data["versions"][0]["shots"][shot]["opt_points"][t_i]["crf"]
        file_list = file_list + "file '" + DIST_PATH + str(shot) + "/" \
        + str(opt_crf) + "_" + codec.upper() + "." + s_cod["container"] + "' \n"
    with open(OUT_LIST, 'w') as w:
        w.write(file_list)
    o = OUT_PATH+source_name[:9] + "_" + t_name + str(t_val) + "_" + codec.upper() + "." + s_cod["container"]
    mux = f"ffmpeg -f concat -i {OUT_LIST} -c copy {o}"
    subprocess.call(mux, shell=True)

In [7]:
target_index = 0
for t_name in target_list:
    for t_val in target_list[t_name]:
        mux(target_index, t_name, t_val)
        target_index += 1

ffmpeg version N-106635-g83e1a1de88 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/home/ubuntu/ffmpeg_build --pkg-config-flags=--static --extra-cflags=-I/home/ubuntu/ffmpeg_build/include --extra-ldflags=-L/home/ubuntu/ffmpeg_build/lib --extra-libs='-lpthread -lm' --ld=g++ --bindir=/home/ubuntu/bin --enable-gpl --enable-gnutls --enable-libaom --enable-libass --enable-libfdk-aac --enable-libfreetype --enable-libmp3lame --enable-libopus --enable-libsvtav1 --enable-libdav1d --enable-libvorbis --enable-libvpx --enable-libx264 --enable-libx265 --enable-libvmaf --enable-version3 --enable-nonfree
  libavutil      57. 24.101 / 57. 24.101
  libavcodec     59. 26.100 / 59. 26.100
  libavformat    59. 22.100 / 59. 22.100
  libavdevice    59.  6.100 / 59.  6.100
  libavfilter     8. 33.100 /  8. 33.100
  libswscale      6.  6.100 /  6.  6.100
  libswresample   4.  6.100 /  4.  6.100
  libpostproc    56.  5.100 / 56.  5.100

## TODO: Curve fitting
When the upper search has tested 3 points, given these 3 RQ points, discover the polynomian or logarithmic function that describes their trend. Repeat this when a new point is computed. Measure the error between the approximation and the actual implementation (lagrangian search above) and assess whether and when it may be useful to speed up the search process, by reducing the number of test to encode before the optimum.